In [1]:
cd ..

C:\Users\kevin.rose\Documents\GitHub\cjp-article-tagging\lib


In [2]:
import tagnews
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import json
import requests
import keras

Using TensorFlow backend.


In [3]:
glove = tagnews.load_glove('tagnews/data/glove.6B.50d.txt')
ner = tagnews.load_ner_data('tagnews/data/')

b'Skipping line 281837: expected 25 fields, saw 34\n'
C:\Users\kevin.rose\AppData\Local\Continuum\Anaconda2\envs\cjp\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
with open('notebooks/training.txt', encoding='utf-8') as f:
    training_data = f.read()

In [5]:
training_df = pd.DataFrame([x.split() for x in training_data.split('\n')], columns=['word', 'tag'])
training_df.iloc[:,1] = training_df.iloc[:,1].apply(int)
training_df['all_tags'] = 'NA'

ner = training_df#pd.concat([training_df, ner]).reset_index(drop=True)
ner = ner[['word', 'all_tags', 'tag']]

In [6]:
ner = pd.concat([ner,
                 pd.DataFrame(ner['word'].str[0].str.isupper().values),
                 pd.DataFrame(glove.loc[ner['word'].str.lower()].values)],
                axis='columns')
ner.fillna(value=0.0, inplace=True)

data_dim = 51
timesteps = 25 # only during training, testing can take arbitrary length.
num_classes = 2

train_val_split = int(19 * ner.shape[0] / 20.)

ner_train_idxs = range(0, train_val_split - timesteps, timesteps)
x_train = np.array([ner.iloc[i:i+timesteps, 3:].values
                    for i in ner_train_idxs])
y_train = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                    for i in ner_train_idxs])

ner_val_idxs = range(train_val_split, ner.shape[0] - timesteps, timesteps)
x_val = np.array([ner.iloc[i:i+timesteps, 3:].values
                  for i in ner_val_idxs])
y_val = np.array([to_categorical(ner.iloc[i:i+timesteps, 2].values, 2)
                  for i in ner_val_idxs])

In [7]:
model = Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=(None, data_dim)))
model.add(LSTM(8, return_sequences=True))
model.add(TimeDistributed(Dense(2, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])
print(model.summary(100))

____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
lstm_1 (LSTM)                                (None, None, 32)                        10752          
____________________________________________________________________________________________________
lstm_2 (LSTM)                                (None, None, 8)                         1312           
____________________________________________________________________________________________________
time_distributed_1 (TimeDistributed)         (None, None, 2)                         18             
Total params: 12,082
Trainable params: 12,082
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [8]:
checkpointer = ModelCheckpoint(filepath='./tmp/weights-{epoch:02d}.hdf5',
                               monitor='val_categorical_accuracy',
                               verbose=1,
                               save_best_only=True)

class OurAUC(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        # Go to https://geo-extract-tester.herokuapp.com/ and download
        # the validation data (validation.txt).
        with open('validation.txt', encoding='utf-8') as f:
            s = f.read()

        gloved_data = pd.concat([pd.DataFrame([[w[0].isupper()] for w in s.split('\n') if w]),
                                 glove.loc[[w for w in s.split('\n') if w]].fillna(0).reset_index(drop=True)],
                               axis='columns')
        
        glove_time_size = 100
        preds_batched = []
        i = 0
        while gloved_data[i:i+glove_time_size].size:
            preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size],
                                                              axis=0))[0][:,1])
            i += glove_time_size

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'w') as f:
            for prob in [p for pred in preds_batched for p in pred]:
                f.write(str(prob) + '\n')

        with open('guesses-{epoch:02d}.txt'.format(epoch=epoch), 'rb') as f:
            url = 'https://geo-extract-tester.herokuapp.com/api/score'
            r = requests.post(url, files={'file': f})
            print('AUC: {:.5f}'.format(json.loads(r.text)['auc']))

our_auc = OurAUC()

In [9]:
model.fit(x_train, y_train,
          epochs=20,
          validation_data=(x_val, y_val),
          callbacks=[checkpointer, our_auc],
          verbose=2)

Train on 2467 samples, validate on 129 samples
Epoch 1/20
Epoch 00001: val_categorical_accuracy improved from -inf to 0.92992, saving model to ./tmp/weights-01.hdf5
AUC: 0.88242
 - 16s - loss: 0.3766 - categorical_accuracy: 0.8850 - val_loss: 0.2403 - val_categorical_accuracy: 0.9299
Epoch 2/20
Epoch 00002: val_categorical_accuracy improved from 0.92992 to 0.92992, saving model to ./tmp/weights-02.hdf5
AUC: 0.92473
 - 13s - loss: 0.2043 - categorical_accuracy: 0.9165 - val_loss: 0.1696 - val_categorical_accuracy: 0.9299
Epoch 3/20
Epoch 00003: val_categorical_accuracy did not improve
AUC: 0.93838
 - 13s - loss: 0.1567 - categorical_accuracy: 0.9391 - val_loss: 0.1575 - val_categorical_accuracy: 0.9253
Epoch 4/20
Epoch 00004: val_categorical_accuracy did not improve
AUC: 0.94186
 - 12s - loss: 0.1458 - categorical_accuracy: 0.9423 - val_loss: 0.1501 - val_categorical_accuracy: 0.9290
Epoch 5/20
Epoch 00005: val_categorical_accuracy did not improve
AUC: 0.94330
 - 10s - loss: 0.1395 - ca

In [13]:
idx = slice(501, 550)
print(pd.concat([ner.iloc[idx, :3].reset_index(drop=True),
                 pd.DataFrame(model.predict(np.expand_dims(ner.iloc[idx, 3:].values, 0))[0][:, 1:],
                              columns=['prob_geloc'])],
                axis='columns'))

# Go to https://geo-extract-tester.herokuapp.com/ and download
# the validation data (validation.txt).
with open('validation.txt', encoding='utf-8') as f:
    s = f.read()
    
gloved_data = glove.loc[[w for w in s.split('\n') if w]].fillna(0)
gloved_data.insert(0, 'case', [w[0].isupper() for w in s.split('\n') if w])
glove_time_size=100
preds_batched = []
i = 0
while gloved_data[i:i+glove_time_size].size:
    preds_batched.append(model.predict(np.expand_dims(gloved_data[i:i+glove_time_size], axis=0))[0][:,1])
    i += glove_time_size

with open('guesses.txt', 'w') as f:
    for prob in [p for pred in preds_batched for p in pred]:
        f.write(str(prob) + '\n')

# Now go to https://geo-extract-tester.herokuapp.com/ and upload `guesses.txt` to see how you did!

           word all_tags  tag  prob_geloc
0            to       NA    0    0.032528
1             a       NA    0    0.002263
2          pool       NA    0    0.001998
3            of       NA    0    0.002051
4        reward       NA    0    0.001419
5         money       NA    0    0.000777
6           for       NA    0    0.000653
7   information       NA    0    0.000576
8       leading       NA    0    0.000635
9            to       NA    0    0.000730
10          the       NA    0    0.000955
11       gunman       NA    0    0.000850
12            ’       NA    0    0.001382
13            s       NA    0    0.002142
14       arrest       NA    0    0.001075
15            .       NA    0    0.001074
16          Ald       NA    0    0.003206
17            .       NA    0    0.001322
18         Will       NA    0    0.001027
19        Burns       NA    0    0.001338
20            ,       NA    0    0.001105
21           in       NA    0    0.001440
22        whose       NA    0    0